# Import Libraries

In [ ]:
#DataFrames manipulation
import pandas as pd

#System Libraries
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta, FR
import os
import shutil

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim

#Libraries to save the plots to html object
import panel as pn

hv.extension('bokeh')

# Yahoo Finance Download

In [ ]:
from fake_useragent import UserAgent
from yahooquery import Ticker
from tqdm import tqdm

In [ ]:
#Create a Data Folder
Data_folder = os.path.abspath(os.getcwd() +'/Data/')
if not os.path.exists(Data_folder):
    os.makedirs(Data_folder)

#Clean older files and folders in the Data Folder
filelist = [ f for f in os.listdir(Data_folder)]
for f in filelist:
    shutil.rmtree(os.path.join(Data_folder, f), ignore_errors=True)

filelist = [ f for f in os.listdir(Data_folder)]
for f in filelist:
    os.remove(os.path.join(Data_folder, f))

#Create New Folder for Stock Indices
stock_indices_folder = os.path.join(Data_folder, 'Stock Indices')
if not os.path.exists(stock_indices_folder):
    os.makedirs(stock_indices_folder)

In [ ]:
#Instantiate the Indices data
indices_data = pd.read_csv('Sector_Indices.csv') 

#Create Indices variables and Manage the Column Data to create new variables
indices_tickers = indices_data['Yahoo Symbol']

last_date = datetime.now() + relativedelta(weekday=FR(-1)) #last friday
init_date = (last_date - timedelta(days=366*50)).strftime("%Y-%m-%d")
frequency = '1d' #Author prefers weekly; '1d' for daily

In [ ]:
#Run code for the tickers in Index Tickers list
for ticker in tqdm(indices_tickers):
    try:

        #Download Ticker Price Data
        ua = UserAgent()
        stock = Ticker(ticker, user_agent=ua.random)
        price_table = stock.history(start = init_date, interval=frequency)
        price_table = price_table.reset_index()
        price_table = price_table.rename(columns = {'open':ticker})
        price_table = price_table.set_index('date')
        price_table = pd.DataFrame(price_table[ticker])

        #Check Ticker currency - In case is not USD convert to USD
        currency = stock.price[ticker]['currency']
        if currency == 'USD':
            price_table = price_table
            price_table.to_csv(os.path.join(stock_indices_folder,ticker)+'_Table.csv')
        elif currency != 'currency':
            currency = stock.price[ticker]['currency']
            currency_symbol = currency + 'USD=X'

            ua = UserAgent()
            exchange_rate = Ticker(currency_symbol, user_agent=ua.random)
            exchange_rate_table = exchange_rate.history(start = init_date , interval='1d')
            exchange_rate_table = exchange_rate_table.reset_index()
            exchange_rate_table = exchange_rate_table.rename(columns = {'open':currency})
            exchange_rate_table = exchange_rate_table.set_index('date')
            exchange_rate_table = pd.DataFrame(exchange_rate_table[currency])

            #Merge the Ticker Price Table with the Exchange rate Table
            price_table = price_table.merge(exchange_rate_table, on ='date')
            price_table = price_table.multiply(price_table[currency], axis=0) #Convert to USD
            price_table = price_table.round(2).drop(columns=currency)
            price_table.to_csv(os.path.join(stock_indices_folder,ticker)+'_Table.csv')
    except:
        continue

In [ ]:
#Join All the Ticker Data Files into a Table
files= os.listdir(stock_indices_folder)

indices_data_table = pd.read_csv(os.path.join(stock_indices_folder,files[0]))

for file in files[1:]:
    df = pd.read_csv(os.path.join(stock_indices_folder,file)) 
    indices_data_table = indices_data_table.merge(df, on='date', how='left')

#Rearrange the DataFrame format: fill na's, rounding, etc.
indices_data_table = indices_data_table.fillna(method='bfill')
indices_data_table = indices_data_table.round(2)
indices_data_table = indices_data_table.set_index('date')
indices_data_table = indices_data_table.head(-1)

indices_data_table.to_csv(os.path.join(Data_folder,'Stock_Indices_Data_Table.csv'))

In [ ]:
#Convert to Monthly Data
indices_data_table.index = pd.to_datetime(indices_data_table.index)
indices_data_table = indices_data_table.resample('M').last()

#Drop Last row
indices_data_table = indices_data_table[:-1]

#Get Percentage Change
indices_data_table = indices_data_table.pct_change()*100
indices_data_table = indices_data_table[1:]

#Rename Headers from Indices' Tickers to Names
indices_data_table = indices_data_table.rename(columns=dict(zip(indices_data['Yahoo Symbol'], indices_data['Name'])))

# Manipulate the Data for the Heatmap

In [ ]:
#Limit the Table to S&P 500 Index
spx_df = pd.DataFrame(indices_data_table['S&P 500 Index'])

#Drop the NA's and Reset Index
spx_df = spx_df.replace([np.inf, -np.inf], np.nan).dropna()
spx_df = spx_df.reset_index()

#Limit the Dataframe to start at the beginning of 1988
define_first_value = spx_df[spx_df['date'] == '1988-01-31'].index[0]
spx_df = spx_df[define_first_value:]

#Create New columns with month and year for each row
spx_df['year'] = pd.DatetimeIndex(spx_df['date']).year
spx_df['month'] = pd.DatetimeIndex(spx_df['date']).month

#Convert Month from Integer to 3-letter name
spx_df['month'] = pd.to_datetime(spx_df['month'], format='%m').dt.month_name().str.slice(stop=3)

In [ ]:
#Convert table to Matrix Format
spx_df = spx_df.groupby(['year','month']).mean().unstack()
spx_df.columns = spx_df.columns.get_level_values(1)
spx_df = spx_df[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]

#Sort table by years ascending
spx_df = spx_df.sort_index(ascending=False)

#Round values t 2 decimal cases
spx_df = spx_df.round(2)

In [ ]:
# Create a New Table with Statistics: Average Max, Mean, %Times Positive, %Times Negative
spx_stats_df = spx_df[0:0]
spx_stats_df.loc['% Neg.'] = (spx_df<0).sum() / (spx_df.count())*100
spx_stats_df.loc['% Pos.'] = (spx_df>=0).sum() / (spx_df.count())*100
spx_stats_df.loc['Min.'] = spx_df.min()
spx_stats_df.loc['Avg.'] = spx_df.mean()
spx_stats_df.loc['Max.'] = spx_df.max()

spx_stats_df = spx_stats_df.round(2)

In [ ]:
#Convert the table to a style where Heatmap works
spx_stats_df_hv = spx_stats_df.reset_index()
spx_stats_df_hv = pd.melt(spx_stats_df_hv, id_vars='year', var_name='Month', value_name='Returns')
spx_stats_df_hv = spx_stats_df_hv[['Month','year','Returns']]

# Creating the HeatMap

In [ ]:
#Define the Grids within the Heatmap
grid_style = {'grid_line_color': 'black', 'grid_line_width': 100}

#Define the Frame around the Heatmap
def hook(plot, element):
    plot.state.outline_line_width = 2
    plot.state.outline_line_color="black"

heatmap_rows_list = ['% Pos.', 'Min.', 'Max.', 'Avg.']

heatmaps = []

for heatmap_row in heatmap_rows_list:
    data = spx_stats_df_hv[spx_stats_df_hv['year'] == heatmap_row]
    heatmap = hv.HeatMap(data, label=f"Year {heatmap_row}")
    heatmap = heatmap.opts(
        opts.HeatMap(width=700, height=115, xrotation=45, xaxis='top', labelled=[],
                     tools=['hover'], cmap='RdYlGn',
                     fontsize={'title': 15, 'xticks': 10, 'yticks': 10},
                     ))
    heatmap = heatmap.opts(gridstyle=grid_style, show_grid=True, hooks=[hook])
    heatmap = heatmap * hv.Labels(heatmap).opts(padding=0)
    
    heatmaps.append(heatmap)
    
overlayed_heatmap = hv.Overlay(heatmaps, label="S&P 500 Index - Monthly Seasonality Statistics (1988-2023)").opts(opts.Overlay(show_legend=False, height=300, ))


In [ ]:
heatmap_spx_stats_panel = pn.panel(overlayed_heatmap)
heatmap_spx_stats_panel.save('Heatmap_SPX_Stats.html')